In [1]:
import pandas as pd

In [2]:
# input = """1
# 10
# 100
# 2024"""

# input = """1
# 2
# 3
# 2024"""


input = open("inputs/22").read()

In [3]:
secret_numbers = [int(i) for i in input.split()]

In [4]:
def mix(secret, value):
    return secret ^ value


def prune(secret):
    # 2**24
    return secret % 16777216


def op1(secret):
    # 2**6
    return prune(mix(secret, secret * 64))


def op2(secret):
    # 2**5
    return prune(mix(secret, secret // 32))


def op3(secret):
    # 2**11
    return prune(mix(secret, secret * 2048))

In [5]:
2**13, 2**14

(8192, 16384)

In [6]:
def next_secret(secret):
    return op3(op2(op1(secret)))


def iterate_secret(secret, n):
    for _ in range(n):
        secret = next_secret(secret)
        # print(secret)

    return secret


def iterate_secret_with_memory(secret, n):
    memory = [secret]

    for _ in range(n):
        memory.append(next_secret(memory[-1]))

    return memory

In [7]:
sum(iterate_secret(n, 2000) for n in secret_numbers)

18317943467

In [8]:
18**4

104976

In [9]:
rows = []

for secret_number in secret_numbers:
    evolution = iterate_secret_with_memory(secret_number, n=2000)

    rows.extend(
        dict(secret_number=secret_number, i=i, evolution=e)
        for i, e in enumerate(evolution, start=0)
    )

In [10]:
evolution_df = pd.DataFrame(rows)

In [11]:
(evolution_df.groupby("evolution").size() > 1).sum()

np.int64(470961)

In [12]:
evolution_df["evolution_digit"] = evolution_df["evolution"].apply(lambda x: x % 10)

In [13]:
evolution_df["diff"] = evolution_df.groupby("secret_number")["evolution_digit"].diff()

In [14]:
evolution_df["diff"].value_counts()

diff
 0.0    435889
 1.0    392173
-1.0    391558
-2.0    348770
 2.0    348445
 3.0    305813
-3.0    305567
 4.0    261883
-4.0    261606
-5.0    218165
 5.0    217621
-6.0    173722
 6.0    173437
-7.0    130657
 7.0    130129
 8.0     87866
-8.0     87202
 9.0     43810
-9.0     43687
Name: count, dtype: int64

In [15]:
import numpy as np

evolution_df = evolution_df.dropna().assign(diff=lambda x: x["diff"].astype(np.int64))

In [16]:
evolution_df

,secret_number,i,evolution,evolution_digit,diff
1,10091473,1,6250477,7,4
2,10091473,2,8429960,0,-7
3,10091473,3,8463764,4,4
4,10091473,4,14320272,2,-2
5,10091473,5,12900260,0,-2
...,...,...,...,...,...
4360174,12126945,1996,2795262,2,-7
4360175,12126945,1997,9087413,3,1
4360176,12126945,1998,3625682,2,-1
4360177,12126945,1999,3144032,2,0


In [17]:
# First create the shifted columns
evolution_df["digit_1"] = evolution_df.groupby("secret_number")["diff"].shift(3)
evolution_df["digit_2"] = evolution_df.groupby("secret_number")["diff"].shift(2)
evolution_df["digit_3"] = evolution_df.groupby("secret_number")["diff"].shift(1)

# Drop rows with NaN values in any of the digit columns
clean_df = evolution_df.dropna(subset=["digit_1", "digit_2", "digit_3", "diff"]).copy()

# Convert to integers before string concatenation
clean_df["last4_digits_concat"] = (
    clean_df["digit_1"].astype(int).astype(str)
    + clean_df["digit_2"].astype(int).astype(str)
    + clean_df["digit_3"].astype(int).astype(str)
    + clean_df["diff"].astype(int).astype(str)
)

In [18]:
clean_df

,secret_number,i,evolution,evolution_digit,diff,digit_1,digit_2,digit_3,last4_digits_concat
4,10091473,4,14320272,2,-2,4.0,-7.0,4.0,4-74-2
5,10091473,5,12900260,0,-2,-7.0,4.0,-2.0,-74-2-2
6,10091473,6,4996945,5,5,4.0,-2.0,-2.0,4-2-25
7,10091473,7,14924873,3,-2,-2.0,-2.0,5.0,-2-25-2
8,10091473,8,5702521,1,-2,-2.0,5.0,-2.0,-25-2-2
...,...,...,...,...,...,...,...,...,...
4360174,12126945,1996,2795262,2,-7,2.0,-1.0,6.0,2-16-7
4360175,12126945,1997,9087413,3,1,-1.0,6.0,-7.0,-16-71
4360176,12126945,1998,3625682,2,-1,6.0,-7.0,1.0,6-71-1
4360177,12126945,1999,3144032,2,0,-7.0,1.0,-1.0,-71-10


In [19]:
from collections import defaultdict

code_dicts = []

for group_name, group_df in clean_df.groupby("secret_number"):
    print(f"Processing group: {group_name}")
    dct = {}

    # for v in group_df['last_4_digits_concat']:
    for i, row in group_df.iterrows():
        last_4 = row["last4_digits_concat"]

        if last_4 in dct:
            continue

        dct[last_4] = row["evolution_digit"]

    code_dicts.append(dct)

Processing group: 162956
Processing group: 178718
Processing group: 188556
Processing group: 199829
Processing group: 209572
Processing group: 226806
Processing group: 228429
Processing group: 237889
Processing group: 262311
Processing group: 262449
Processing group: 267035
Processing group: 274021
Processing group: 275156
Processing group: 288309
Processing group: 290940
Processing group: 305655
Processing group: 314863
Processing group: 329177
Processing group: 332014
Processing group: 333198
Processing group: 335756
Processing group: 342827
Processing group: 348147
Processing group: 360811
Processing group: 387798
Processing group: 411106
Processing group: 425187
Processing group: 447934
Processing group: 452998
Processing group: 453314
Processing group: 456270
Processing group: 459568
Processing group: 493276
Processing group: 510480
Processing group: 522860
Processing group: 535812
Processing group: 535831
Processing group: 539585
Processing group: 540230
Processing group: 548271


In [20]:
len(code_dicts)

2179

In [21]:
all_keys = set()
for dct in code_dicts:
    all_keys.update(dct.keys())
len(all_keys)

40951

In [22]:
key_total_values = defaultdict(int)
for dct in code_dicts:
    for key, value in dct.items():
        key_total_values[key] += value
key_total_values

defaultdict(int,
            {'-6112': 757,
             '1123': 1123,
             '123-5': 401,
             '23-54': 1369,
             '3-54-4': 387,
             '-54-40': 431,
             '4-402': 1190,
             '-402-3': 423,
             '02-34': 1662,
             '2-342': 1203,
             '-3420': 1208,
             '420-3': 890,
             '20-3-2': 450,
             '0-3-26': 1185,
             '-3-26-3': 740,
             '-26-31': 932,
             '6-311': 971,
             '-311-7': 41,
             '11-79': 477,
             '1-79-7': 82,
             '-79-71': 126,
             '9-711': 180,
             '-711-3': 50,
             '11-35': 1483,
             '1-350': 1386,
             '-350-6': 253,
             '50-68': 648,
             '0-681': 378,
             '-681-5': 172,
             '81-51': 245,
             '1-510': 723,
             '-510-5': 0,
             '10-52': 797,
             '0-521': 1013,
             '-521-2': 591,
             '21-2

In [23]:
# 4149 is too high
max(key_total_values.values())

2018